In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from phase_generator import *

Initalize simulation parameters and the simulator

In [ ]:
# Parameters
D  = 8.0  # Size of the phase screen [m]
r0 = 0.1  # Fried parameter [m]
L0 = 25.0 # Outer scale [m]

dx = r0 / 3.0 # Spatial sampling interval [m/pixel], make sure r0 is Nyquist sampled
dt = 0.001 # Time step [s/step]

wind_speed = 40 # [m/s]
wind_direction = 45 # [degree]
boiling_factor = 0*500 # [a.u], need to figure them out

screen_generator = CascadedPhaseGenerator(D, dx, dt, batch_size=config['batch_size'], n_cascades=3)
screen_generator.AddLayer(0.5, r0, L0, wind_speed, wind_direction, boiling_factor)
screen_generator.AddLayer(0.5, r0/2., L0, wind_speed*2, wind_direction*4, boiling_factor)

Generate 1000 atmospheric phase screens. Even though the phase screens here are accesses one by one, the batch with $N_{screens}$ is pre-generated. Then, the screens are read from it.

In [ ]:
if GPU_flag:
    start = cp.cuda.Event()
    end   = cp.cuda.Event()

total_time = 0
screens_cascade = []

for i in tqdm(range(1000)):
    if GPU_flag:
        start.record()
    else:
        start = time.time()
        
    screens_cascade.append(screen_generator.GetScreenByTimestep(i))
    
    if GPU_flag:
        end.record()
        end.synchronize()
        total_time += cp.cuda.get_elapsed_time(start, end)  # Time in [ms]
    else:
        end = time.time()
        total_time += (end-start) * 1000

screens_cascade = xp.dstack(screens_cascade)

print(f"Total elapsed time: {np.round(total_time/1e3,1)} [s]")
print(f"Time per sreen: {np.round(total_time/screen_generator.num_screens).astype(xp.uint16)} [ms]")

Save as video file

In [ ]:
SaveVideo(screens_cascade)

Compare the PSD estimated from the series of previously generated screens with a theoretical oversampled PSD

In [ ]:
oversampling_factor = 17
_,_,f_over, df_over = screen_generator.freq_array(screen_generator.N*oversampling_factor, dx)

rad2nm = 500.0 / 2.0 / np.pi # [nm/rad]

PSD_ultimate = screen_generator.vonKarmanPSD(f_over, r0, L0) * df_over**2 * rad2nm**2 * oversampling_factor**2

PSD_out = PSD_to_phase(screens_cascade)
if GPU_flag:
    PSD_out = cp.asnumpy(PSD_out)


radialize_PSD(PSD_ultimate, f_over, 'Ultimate PSD')
radialize_PSD(PSD_out, screen_generator.f[...,0], 'Reconstructed PSD')

plt.grid()
plt.title('Theoretical oversampled PSD vs. PSD from phases')
plt.legend()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Spatial frequency [1/m]')
plt.ylabel(r'$PSD [nm^2/m^2]$')
plt.show()